In [1]:
import pandas as pd
import config
from binance.client import Client

In [2]:
api_key = config.api_key
api_secret = config.api_secret

client = Client(api_key, api_secret)

## BTC-USDC Spot Market Data

In [3]:
# Function to fetch spot market data
def fetch_binance_spot_data(symbol, interval, start_str, end_str):
    klines = client.get_historical_klines(symbol, interval, start_str, end_str)
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    return df

In [4]:
# Parameters
symbol_spot = 'BTCUSDC'
interval_spot = Client.KLINE_INTERVAL_1DAY
start_str_spot = '2021-01-01'
end_str_spot = '2024-05-31'

In [5]:
# Fetch the data and save to CSV
df_spot = fetch_binance_spot_data(symbol_spot, interval_spot, start_str_spot, end_str_spot)
df_spot.to_csv('data/btc_usdc_spot_data.csv')
print(df_spot.head())

                open      high       low     close       volume    close_time  \
timestamp                                                                       
2021-01-01  28964.54  29680.00  28608.73  29407.93  1736.620480  1.609546e+12   
2021-01-02  29393.99  33500.00  29027.03  32215.18  4227.234681  1.609632e+12   
2021-01-03  32216.13  34851.30  32002.65  33033.26  3523.881120  1.609718e+12   
2021-01-04  33040.00  33648.27  28000.00  32017.58  4218.572357  1.609805e+12   
2021-01-05  32027.11  34498.00  29882.30  34059.70  3150.244271  1.609891e+12   

            quote_asset_volume  number_of_trades  taker_buy_base_asset_volume  \
timestamp                                                                       
2021-01-01        5.084403e+07           28084.0                   782.686746   
2021-01-02        1.327970e+08           51652.0                  2378.789351   
2021-01-03        1.179675e+08           52963.0                  1907.369385   
2021-01-04        1.327577e

## BTC-USDC Futures Market Data

In [6]:
# Function to fetch futures market data
def fetch_binance_futures_data(client, symbol, interval, start_str, end_str):
    klines = client.futures_historical_klines(symbol, interval, start_str, end_str)
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    return df

In [7]:
# Parameters
symbol_futures = 'BTCUSDC'
interval_futures = Client.KLINE_INTERVAL_1DAY  # 1-day interval
start_str_futures = '2021-01-01'
end_str_futures = '2024-05-31'

In [8]:
# Fetch the data
df_futures = fetch_binance_futures_data(client, symbol_futures, interval_futures, start_str_futures, end_str_futures)
df_futures.to_csv('data/btc_usdc_futures_data.csv')
print(df_futures.head())

                open      high       low     close       volume    close_time  \
timestamp                                                                       
2021-01-01  28964.54  29680.00  28608.73  29407.93  1736.620480  1.609546e+12   
2021-01-02  29393.99  33500.00  29027.03  32215.18  4227.234681  1.609632e+12   
2021-01-03  32216.13  34851.30  32002.65  33033.26  3523.881120  1.609718e+12   
2021-01-04  33040.00  33648.27  28000.00  32017.58  4218.572357  1.609805e+12   
2021-01-05  32027.11  34498.00  29882.30  34059.70  3150.244271  1.609891e+12   

            quote_asset_volume  number_of_trades  taker_buy_base_asset_volume  \
timestamp                                                                       
2021-01-01        5.084403e+07           28084.0                   782.686746   
2021-01-02        1.327970e+08           51652.0                  2378.789351   
2021-01-03        1.179675e+08           52963.0                  1907.369385   
2021-01-04        1.327577e

## BTC Funding Rate Data

In [9]:
# Function to fetch funding rate data
def fetch_binance_funding_rate(client, symbol, start_str, end_str=None):
    df_list = []
    start_time = pd.to_datetime(start_str)
    end_time = pd.to_datetime(end_str) if end_str else pd.Timestamp.now()

    while start_time < end_time:
        funding_rates = client.futures_funding_rate(
            symbol=symbol,
            startTime=int(start_time.timestamp() * 1000),
            endTime=int(end_time.timestamp() * 1000),
            limit=1000
        )
        if not funding_rates:
            break
        df = pd.DataFrame(funding_rates)
        df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
        df_list.append(df)
        start_time = df['fundingTime'].iloc[-1] + pd.Timedelta(milliseconds=1)  # Update start_time to the last fetched timestamp
    
    all_data = pd.concat(df_list)
    all_data.set_index('fundingTime', inplace=True)
    return all_data

# Function to calculate daily average funding rate
def calculate_daily_average_funding_rate(df):
    df_daily_avg = df.resample('D').mean()
    return df_daily_avg

In [10]:
# Parameters
symbol_funding = 'BTCUSDT'
start_str_funding = '2021-01-01'
end_str_funding = '2024-05-31'

In [11]:
# Fetch the data
df_funding = fetch_binance_funding_rate(client, symbol_funding, start_str_funding, end_str_funding)

# Convert relevant columns to float
numeric_columns = ['fundingRate']
df_funding[numeric_columns] = df_funding[numeric_columns].astype(float)

# Calculate daily average funding rate
df_daily_avg_funding = calculate_daily_average_funding_rate(df_funding)

# Save to CSV
df_daily_avg_funding.to_csv('data/btc_usdt_daily_avg_funding_rate.csv')
print(df_daily_avg_funding.head())

             fundingRate
fundingTime             
2021-01-01      0.000278
2021-01-02      0.000485
2021-01-03      0.001313
2021-01-04      0.001522
2021-01-05      0.000661


/var/folders/rc/by_5fr2d2ds3vzb0s0qcn32r0000gn/T/ipykernel_61252/3699113751.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_daily_avg = df.resample('D').mean()


## USDC Price Data

In [12]:
# Parameters for USDC price
symbol_usdc = 'USDCUSDT'
interval_usdc = Client.KLINE_INTERVAL_1DAY
start_str_usdc = '2021-01-01'
end_str_usdc = '2024-05-31'

In [13]:
# Fetch the data and save to CSV
df_usdc = fetch_binance_spot_data(symbol_usdc, interval_usdc, start_str_usdc, end_str_usdc)
df_usdc.to_csv('data/usdc_usdt_price.csv')
print(df_usdc.head())

              open    high     low   close        volume    close_time  \
timestamp                                                                
2021-01-01  0.9986  0.9986  0.9966  0.9974  6.768929e+07  1.609546e+12   
2021-01-02  0.9974  1.0009  0.9970  0.9989  1.515834e+08  1.609632e+12   
2021-01-03  0.9990  1.0011  0.9898  0.9992  1.907760e+08  1.609718e+12   
2021-01-04  0.9991  1.0014  0.9891  0.9994  2.186078e+08  1.609805e+12   
2021-01-05  0.9994  0.9998  0.9961  0.9969  1.301291e+08  1.609891e+12   

            quote_asset_volume  number_of_trades  taker_buy_base_asset_volume  \
timestamp                                                                       
2021-01-01        6.754410e+07           41921.0                 3.145614e+07   
2021-01-02        1.513376e+08           61430.0                 7.562928e+07   
2021-01-03        1.904469e+08           79609.0                 1.006859e+08   
2021-01-04        2.182290e+08           96742.0                 1.130549e+0

## BTC VIX Data

In [14]:
# Calculate daily returns
df_futures['daily_return'] = df_futures['close'].pct_change()

# Calculate rolling volatility (standard deviation of daily returns)
df_futures['volatility'] = df_futures['daily_return'].rolling(window=30).std() * (365**0.5)

# Save the volatility data to CSV
df_futures[['volatility']].dropna().to_csv('data/btc_usdc_volatility.csv')
print(df_futures[['volatility']].head())

            volatility
timestamp             
2021-01-01         NaN
2021-01-02         NaN
2021-01-03         NaN
2021-01-04         NaN
2021-01-05         NaN
